In [1]:
import numpy as np
import pandas as pd
import string
from gensim.models import KeyedVectors

import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

Load word2vec model

In [2]:
model = KeyedVectors.load_word2vec_format(
    'https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz', binary=True)

News text embedding

In [3]:
READ_CSV_PATH = '../dataset/recovery-news-data.csv'
WRITE_TXT_PATH = '../dataset/text_embedding.txt'

df = pd.read_csv(READ_CSV_PATH)
titles = df['title'].values
titles = np.array(titles, dtype=str)
bodies = df['body_text'].values
bodies = np.array(bodies, dtype=str)

MAX_WORD_NUM = 40
NEWS_NUM = len(titles)
print(NEWS_NUM)

news_embedding = np.zeros((NEWS_NUM, MAX_WORD_NUM*300),dtype=float)
print(news_embedding.shape)
# # test
# nid = 0
# text = titles[nid] + bodies[nid]
# puns = string.punctuation + '’“”'
# for ch in puns:
#     text = text.replace(ch, '')
# text = text.replace('\n', ' ')
# text = text.lower()
# words = text.split(' ')
# 
# wid = 0
# num = 0
# for wid in range(len(words)):
#     word = words[wid]
#     if word in model:
#         print(wid, num)
# 
#         vec = model[word]
#         num += 1
#         if num == MAX_WORD_NUM:
#             break

for nid in range(NEWS_NUM):
    if titles[nid] != 'nan':
        text = titles[nid] + bodies[nid]
    else:
        text = bodies[nid]
    puns = string.punctuation + '’“”'
    for ch in puns:
        text = text.replace(ch, '')
    text = text.replace('\n', ' ')
    text = text.lower()
    words = text.split(' ')

    wid = 0
    num = 0
    for wid in range(len(words)):
        word = words[wid]
        if word in model:
            news_embedding[nid,num*300:(num+1)*300] = model[word]
            num += 1
            if num == MAX_WORD_NUM:
                break
    if nid % 100 == 0:
        print(str(nid) + "news articles have been embedded...")

print("Embedding is finished!")
print(news_embedding[:5,:])

np.savetxt(WRITE_TXT_PATH, news_embedding)


2029
(2029, 12000)
0news articles have been embedded...
100news articles have been embedded...
200news articles have been embedded...
300news articles have been embedded...
400news articles have been embedded...
500news articles have been embedded...
600news articles have been embedded...
700news articles have been embedded...
800news articles have been embedded...
900news articles have been embedded...
1000news articles have been embedded...
1100news articles have been embedded...
1200news articles have been embedded...
1300news articles have been embedded...
1400news articles have been embedded...
1500news articles have been embedded...
1600news articles have been embedded...
1700news articles have been embedded...
1800news articles have been embedded...
1900news articles have been embedded...
2000news articles have been embedded...
Embedding is finished!
[[ 0.08007812  0.10498047  0.04980469 ...  0.00366211  0.04760742
  -0.06884766]
 [-0.23535156  0.00787354  0.31054688 ... -0.1298

Using RNN to predict news credibility

In [5]:
'''
Loading file
'''

READ_TXT_PATH = '../dataset/text_embedding.txt'
news_embedding = np.loadtxt(READ_TXT_PATH)


'''
Output the loss histories to files
'''
class LossHistory(tf.keras.callbacks.Callback):
    def __init__(self):
        super().__init__()
        self.losses = []
        self.aucs = []

    def on_train_begin(self, logs={}):
        pass

    def on_batch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))
        self.aucs.append(logs.get('auc'))

'''
Hyperparameters
'''
RANDOM_SEED = 525
BATCH_SIZE = 4
LEARNING_RATE = 0.0001
EPOCH = 100
HIDDEN_DIM = 16
TIME_STEPS = MAX_WORD_NUM

'''
Preparaing data for prediction
'''
READ_CSV_PATH = '../dataset/recovery-news-data.csv'

# 1: unreliable, 0: reliable
df = pd.read_csv(READ_CSV_PATH)
# labels = -1* df['reliability'].values + 1 
labels = df['reliability'].values 
print(np.array(labels, dtype=int).sum())

# Regulate the format
X = news_embedding.reshape(news_embedding.shape[0], TIME_STEPS, 300)
Y = np.vstack((labels, (np.ones(labels.shape)) - labels))
Y = Y.T

# Separate the data as training and testing data
X_train, X_test, Y_train, Y_true = train_test_split(X, Y, test_size=0.2, random_state=RANDOM_SEED)

train = tf.data.Dataset.from_tensor_slices((X_train, Y_train))
test = tf.data.Dataset.from_tensor_slices((X_test, Y_true))

train = train.shuffle(100).batch(BATCH_SIZE)
test = test.batch(BATCH_SIZE)

'''
Model
'''
model = tf.keras.Sequential([
    # tf.keras.layers.SimpleRNN(HIDDEN_DIM, batch_input_shape=(None, TIME_STEPS, 300)),  # simple RNN
    tf.keras.layers.LSTM(HIDDEN_DIM, batch_input_shape=(None, TIME_STEPS, 300)),       # LSTM
    # tf.keras.layers.GRU(HIDDEN_DIM, batch_input_shape=(None, TIME_STEPS, 300)),          # GRU
    tf.keras.layers.Dense(2, activation='softmax')])

log_dir = '../model/logs/fit/' + 'lstm_16'
# log_dir = '../model/logs/fit/' + 'gru_16'

model.compile(optimizer=tf.keras.optimizers.Adam(LEARNING_RATE),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=[ tf.keras.metrics.AUC() ])

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

history = LossHistory()
model.fit(train, epochs=EPOCH,
          callbacks=[tensorboard_callback, history])

model.evaluate(test)

Y_pred = model.predict_classes(X_test)
Y_true = Y_true[:,1]
# print(Y_pred,Y_true[:,1])
print(classification_report(Y_true, Y_pred, target_names=['reliable', 'unreliable']))

1364
Epoch 1/100
406/406 [==============================] - 10s 25ms/step - loss: 0.6356 - auc_1: 0.6971
Epoch 2/100
406/406 [==============================] - 8s 20ms/step - loss: 0.6110 - auc_1: 0.7563
Epoch 3/100
406/406 [==============================] - 7s 18ms/step - loss: 0.5677 - auc_1: 0.7934
Epoch 4/100
406/406 [==============================] - 6s 16ms/step - loss: 0.4975 - auc_1: 0.8442
Epoch 5/100
406/406 [==============================] - 7s 17ms/step - loss: 0.4661 - auc_1: 0.8645
Epoch 6/100
406/406 [==============================] - 7s 16ms/step - loss: 0.4423 - auc_1: 0.8797
Epoch 7/100
406/406 [==============================] - 6s 16ms/step - loss: 0.4236 - auc_1: 0.8899
Epoch 8/100
406/406 [==============================] - 6s 15ms/step - loss: 0.4034 - auc_1: 0.9006
Epoch 9/100
406/406 [==============================] - 7s 16ms/step - loss: 0.3851 - auc_1: 0.9093
Epoch 10/100
406/406 [==============================] - 6s 14ms/step - loss: 0.3695 - auc_1: 0.9163
Epo